La imagen generada se guarda con el nombre imagen_combinada.bmp

In [ ]:
%%writefile combinacion_secuencial.cpp
#include <opencv2/opencv.hpp>
#include <iostream>



int main() {
    cv::Mat fondo = cv::imread("/content/drive/MyDrive/Demat6/Computo Paralelo/Tarea4/fondo.bmp", cv::IMREAD_COLOR);
    cv::Mat imagen = cv::imread("/content/drive/MyDrive/Demat6/Computo Paralelo/Tarea4/greenscreen.bmp", cv::IMREAD_COLOR);
    cv::Mat mask = cv::imread("/content/drive/MyDrive/Demat6/Computo Paralelo/Tarea4/greenscreenMask.bmp", cv::IMREAD_GRAYSCALE);

    if (fondo.empty() || imagen.empty() || mask.empty()) {
        std::cerr << "Error al cargar las imágenes" << std::endl;
        return -1;
    }

    cv::Mat imgResult = cv::Mat::zeros(imagen.size(), imagen.type());

    for (int y = 0; y < imagen.rows; y++) {
        for (int x = 0; x < imagen.cols; x++) {
            cv::Vec3b b = fondo.at<cv::Vec3b>(y, x);
            cv::Vec3b a = imagen.at<cv::Vec3b>(y, x);
            float alpha = mask.at<uchar>(y, x) / 255.0;

            cv::Vec3b c;
            c[0] = a[0] * alpha + (1 - alpha) * b[0];
            c[1] = a[1] * alpha + (1 - alpha) * b[1];
            c[2] = a[2] * alpha + (1 - alpha) * b[2];

            imgResult.at<cv::Vec3b>(y, x) = c;
        }
    }

    // Guardar la imagen combinada
    cv::imwrite("imagen_combinada.bmp", imgResult);

    return 0;
}


In [ ]:
!g++ combinacion_secuencial.cpp -o combinacion_secuencial `pkg-config --cflags --libs opencv4`

In [ ]:
!./combinacion_secuencial

In [ ]:
%%writefile imagenMG.cpp 

#include <opencv2/opencv.hpp>
#include <cmath>

int main(){

    cv::Mat img = cv::imread("/content/drive/MyDrive/Demat6/Computo Paralelo/Tarea4/pinzas_gray.png", cv::IMREAD_GRAYSCALE); 

    if(img.empty()){
        std::cerr <<"Imagen vacia" << std::endl; 
        return -1; 
    }

    cv::Mat K1 = (cv::Mat_<float>(3, 3) << -1, 0, 1, -2, 0, 2, -1, 0, 1);
    cv::Mat K2 = (cv::Mat_<float>(3, 3) << -1, -2, -1, 0, 0, 0, 1, 2, 1);

    cv::Mat imgX, imgY; 
    
    cv::filter2D(img, imgX, CV_32F, K1);
    cv::filter2D(img, imgY, CV_32F, K2);

    cv::Mat MG = cv::Mat::zeros(img.size(), CV_8U);

    for (int y = 0; y < img.rows; y++) {
        for (int x = 0; x < img.cols; x++) {
            float gx = imgX.at<float>(y, x);
            float gy = imgY.at<float>(y, x);
            MG.at<uchar>(y, x) = static_cast<uchar>(cv::sqrt(gx*gx + gy*gy));
        }
    }

    cv::normalize(MG, MG, 0, 255, cv::NORM_MINMAX, CV_8U);
    cv::imwrite("imagen_MG.bmp", MG);

    

    cv::Mat MGT = cv::Mat::zeros(img.size(), CV_8U);

    int T = 100; 

    for (int y = 0; y < img.rows; y++) {
        for (int x = 0; x < img.cols; x++) {
            float comp = MG.at<uchar>(y, x);
            MGT.at<uchar>(y, x) = static_cast<uchar>((comp > T) ? 255 : 0);
        }
    }


    cv::normalize(MGT, MGT, 0, 255, cv::NORM_MINMAX, CV_8U);
    cv::imwrite("imagen_MGT.bmp", MGT);

    return 0;

}